In [ ]:
# PHASE 1: Load Excel Files from GitHub into DataFrames
# ============================================================================

%pip install openpyxl

import pandas as pd
import requests
from io import BytesIO

GITHUB_REPO = "delveccj/LoveDataPresentation_Feb10_2026"
GITHUB_BRANCH = "main"

YEARS_TO_CONVERT = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]

def load_excel_from_github(year):
    filename = f"{year}-List-of-all-ships-dismantled-all-over-the-world.xlsx"
    url = f"https://raw.githubusercontent.com/{GITHUB_REPO}/{GITHUB_BRANCH}/{filename}"
    print(f"Loading {year}...", end=" ")
    response = requests.get(url, timeout=30)
    if response.status_code == 404:
        print("not found")
        return None
    response.raise_for_status()
    
    # 2015+ files have header on a different row
    if year >= 2015:
        df_peek = pd.read_excel(BytesIO(response.content), dtype=str, engine='openpyxl', nrows=5)
        header_row = 0
        for idx in range(min(5, len(df_peek))):
            row_values = df_peek.iloc[idx].astype(str).str.upper().tolist()
            if any(x in str(row_values) for x in ['IMO', 'NAME', 'TYPE', 'LDT', 'BUILT']):
                header_row = idx + 1
                break
        df = pd.read_excel(BytesIO(response.content), dtype=str, engine='openpyxl', header=header_row)
    else:
        df = pd.read_excel(BytesIO(response.content), dtype=str, engine='openpyxl')
    
    print(f"{len(df):,} rows, {len(df.columns)} cols")
    return df

print("=" * 60)
print("LOADING DATA FROM GITHUB")
print("=" * 60)

dataframes = {}
for year in YEARS_TO_CONVERT:
    df = load_excel_from_github(year)
    if df is not None:
        dataframes[year] = df

print(f"\nLoaded {len(dataframes)} datasets")

In [ ]:
# Explore columns - especially owner-related ones
# ============================================================================

print("=" * 60)
print("COLUMN EXPLORATION")
print("=" * 60)

for year, df in dataframes.items():
    print(f"\n{year}:")
    print(f"  All columns: {list(df.columns)}")

In [ ]:
# PHASE 2: Data Integration Pipeline (with Owner data)
# ============================================================================

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

class ShipbreakingIntegrator:
    """Integrates shipbreaking data including beneficial owner information."""
    
    def __init__(self):
        self.unified_columns = [
            'YEAR', 'IMO', 'NAME', 'TYPE', 'GT', 'LDT', 
            'BUILT', 'LAST_FLAG', 'PLACE', 'COUNTRY',
            'OWNER', 'OWNER_COUNTRY'
        ]
        
        self.location_to_country = {
            'alang': 'India', 'bhavnagar': 'India', 'sosiya': 'India',
            'chittagong': 'Bangladesh', 'bangladesh': 'Bangladesh',
            'gadani': 'Pakistan', 'pakistan': 'Pakistan',
            'aliaga': 'Turkey', 'aliağa': 'Turkey', 'turkey': 'Turkey',
            'ghent': 'Belgium', 'belgium': 'Belgium',
            'guangzhou': 'China', 'shanghai': 'China', 'china': 'China',
            'india': 'India',
        }
    
    def find_column(self, df, candidates):
        """Find the first matching column from a list of candidates."""
        for col in candidates:
            if col in df.columns:
                return df[col]
        return pd.Series([np.nan] * len(df))
    
    def extract_country_from_place(self, place):
        if pd.isna(place) or str(place).strip() == '':
            return np.nan
        place_lower = str(place).strip().lower()
        for loc, country in self.location_to_country.items():
            if loc in place_lower:
                return country
        return np.nan
    
    def process_year(self, year, df_in):
        """Process a single year's dataframe into unified schema."""
        result = pd.DataFrame()
        n_rows = len(df_in)
        
        result['YEAR'] = [year] * n_rows
        
        # IMO
        result['IMO'] = self.find_column(df_in, [
            'IMO#', 'IMO  number', 'IMO number', 'IMO'
        ])
        
        # NAME
        result['NAME'] = self.find_column(df_in, [
            'NAME', 'Name of ship ', 'Name of ship', 'SHIP NAME'
        ])
        
        # TYPE
        result['TYPE'] = self.find_column(df_in, [
            'TYPE', 'Type of ship', 'SHIP TYPE', 'Type'
        ])
        
        # GT
        result['GT'] = self.find_column(df_in, [
            'GT', 'Gross tonnage (GT)', 'GROSS TONNAGE'
        ])
        
        # LDT
        result['LDT'] = self.find_column(df_in, [
            'LDT', 'Ldt (light displacement ton)', 'LDT (TONNES)'
        ])
        
        # BUILT
        result['BUILT'] = self.find_column(df_in, [
            'BUILT', 'Built in (y)', 'BUILD YEAR', 'Year Built'
        ])
        
        # LAST_FLAG
        result['LAST_FLAG'] = self.find_column(df_in, [
            'FLAG', 'LAST FLAG', 'Last flag', 'LAST FLAG (CHANGE FOR BREAKING)'
        ])
        
        # PLACE
        result['PLACE'] = self.find_column(df_in, [
            'PLACE', 'DESTINATION', 'Destination yard', 'Destination city', 'YARD'
        ])
        
        # COUNTRY
        country_col = self.find_column(df_in, [
            'COUNTRY', 'Destination country', 'DEST. COUNTRY'
        ])
        if country_col.isna().all():
            result['COUNTRY'] = result['PLACE'].apply(self.extract_country_from_place)
        else:
            result['COUNTRY'] = country_col
        
        # OWNER (beneficial owner)
        result['OWNER'] = self.find_column(df_in, [
            'Beneficial owner of the ship', 'BENEFICIAL OWNER', 
            'BO', 'OWNER', 'Beneficial Owner'
        ])
        
        # OWNER_COUNTRY
        result['OWNER_COUNTRY'] = self.find_column(df_in, [
            "Beneficial owner's Country ", 'BO Country ', 
            'Country of the beneficial owner', 'BO COUNTRY',
            'OWNER COUNTRY', 'Beneficial Owner Country'
        ])
        
        return result[self.unified_columns]
    
    def integrate(self, dataframes_dict):
        """Integrate all dataframes into a single unified dataset."""
        all_data = []
        
        for year, df_in in sorted(dataframes_dict.items()):
            print(f"Processing {year}...", end=" ")
            processed = self.process_year(year, df_in)
            
            before = len(processed)
            processed = processed.dropna(subset=['IMO', 'NAME'], how='all')
            after = len(processed)
            
            all_data.append(processed)
            print(f"{after:,} records")
        
        unified = pd.concat(all_data, ignore_index=True)
        
        # Convert numeric columns
        for col in ['GT', 'LDT', 'BUILT', 'YEAR']:
            unified[col] = pd.to_numeric(unified[col], errors='coerce')
        
        # Clean text columns
        for col in ['NAME', 'TYPE', 'LAST_FLAG', 'PLACE', 'COUNTRY', 'OWNER', 'OWNER_COUNTRY']:
            unified[col] = unified[col].astype(str).str.strip()
            unified[col] = unified[col].replace(['nan', 'None', '', 'NaN'], np.nan)
        
        return unified

# Run integration
print("=" * 60)
print("DATA INTEGRATION")
print("=" * 60)

integrator = ShipbreakingIntegrator()
df = integrator.integrate(dataframes)

print(f"\nUnified dataset: {len(df):,} records")
print(f"Columns: {list(df.columns)}")
print(f"\nRecords per year:")
print(df['YEAR'].value_counts().sort_index())

In [ ]:
# Check owner data coverage
# ============================================================================

print("=" * 60)
print("OWNER DATA COVERAGE")
print("=" * 60)

owner_coverage = df.groupby('YEAR').agg({
    'OWNER': lambda x: x.notna().sum(),
    'OWNER_COUNTRY': lambda x: x.notna().sum(),
    'NAME': 'count'
}).rename(columns={'NAME': 'TOTAL'})

owner_coverage['OWNER_PCT'] = (owner_coverage['OWNER'] / owner_coverage['TOTAL'] * 100).round(1)
owner_coverage['OWNER_COUNTRY_PCT'] = (owner_coverage['OWNER_COUNTRY'] / owner_coverage['TOTAL'] * 100).round(1)

print("\nOwner data availability by year:")
print(owner_coverage)

print(f"\nTotal records with OWNER: {df['OWNER'].notna().sum():,} ({df['OWNER'].notna().sum()/len(df)*100:.1f}%)")
print(f"Total records with OWNER_COUNTRY: {df['OWNER_COUNTRY'].notna().sum():,} ({df['OWNER_COUNTRY'].notna().sum()/len(df)*100:.1f}%)")

In [ ]:
# EDA Setup
# ============================================================================

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("="*80)
print("EXPLORATORY DATA ANALYSIS")
print("="*80)
print(f"\nDataset: {len(df):,} records")
print(f"Years: {int(df['YEAR'].min())} - {int(df['YEAR'].max())}")

print("\nMissing data summary:")
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(1)
missing_df = pd.DataFrame({
    'Missing': missing,
    'Pct': missing_pct
})
print(missing_df)

In [ ]:
# ANALYSIS 1: Ships Dismantled Per Year
# ============================================================================

print("\n" + "="*80)
print("ANALYSIS 1: SHIPS DISMANTLED PER YEAR")
print("="*80)

yearly_counts = df['YEAR'].value_counts().sort_index()

fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.bar(yearly_counts.index, yearly_counts.values, color='steelblue', alpha=0.8, edgecolor='black')
ax.set_xlabel('Year', fontsize=12, fontweight='bold')
ax.set_ylabel('Number of Ships', fontsize=12, fontweight='bold')
ax.set_title('Ships Dismantled Per Year (2012-2024)', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

for bar, count in zip(bars, yearly_counts.values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 20, 
            str(count), ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print(f"\nTotal ships dismantled: {len(df):,}")
print(f"Average per year: {yearly_counts.mean():.0f}")
print(f"Peak: {int(yearly_counts.idxmax())} ({yearly_counts.max()} ships)")

In [ ]:
# ANALYSIS 2: Geographic Distribution (Destination Countries)
# ============================================================================

print("\n" + "="*80)
print("ANALYSIS 2: DESTINATION COUNTRIES")
print("="*80)

country_counts = df['COUNTRY'].value_counts().head(10)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart
colors = plt.cm.Reds(np.linspace(0.3, 0.9, len(country_counts)))
ax1.barh(range(len(country_counts)), country_counts.values, color=colors, edgecolor='black')
ax1.set_yticks(range(len(country_counts)))
ax1.set_yticklabels(country_counts.index)
ax1.set_xlabel('Number of Ships', fontweight='bold')
ax1.set_title('Top 10 Ship Dismantling Destinations', fontweight='bold', fontsize=14)
ax1.invert_yaxis()

for i, (count, pct) in enumerate(zip(country_counts.values, country_counts.values/len(df)*100)):
    ax1.text(count + 50, i, f'{count:,} ({pct:.1f}%)', va='center', fontsize=10)

# Pie chart - South Asia vs Others
south_asia = ['India', 'Bangladesh', 'Pakistan']
south_asia_count = df[df['COUNTRY'].isin(south_asia)].shape[0]
other_count = len(df) - south_asia_count

ax2.pie([south_asia_count, other_count], 
        labels=['South Asia\n(India, Bangladesh, Pakistan)', 'Rest of World'],
        autopct='%1.1f%%', colors=['#d62728', '#1f77b4'], startangle=90,
        explode=[0.05, 0], textprops={'fontsize': 12})
ax2.set_title('Regional Distribution of Ship Breaking', fontweight='bold', fontsize=14)

plt.tight_layout()
plt.show()

print(f"\nSouth Asian yards handle {south_asia_count/len(df)*100:.1f}% of global ship breaking")

In [ ]:
# ANALYSIS 3: Ship Types and Ages
# ============================================================================

print("\n" + "="*80)
print("ANALYSIS 3: SHIP CHARACTERISTICS")
print("="*80)

# Ship types
type_counts = df['TYPE'].value_counts().head(10)

# Age calculation
df['AGE'] = df['YEAR'] - df['BUILT']
df_age = df[(df['AGE'] >= 0) & (df['AGE'] <= 80)]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Ship types
ax1.barh(range(len(type_counts)), type_counts.values, color='teal', alpha=0.8)
ax1.set_yticks(range(len(type_counts)))
ax1.set_yticklabels(type_counts.index)
ax1.set_xlabel('Number of Ships', fontweight='bold')
ax1.set_title('Most Common Ship Types Dismantled', fontweight='bold', fontsize=14)
ax1.invert_yaxis()

# Age distribution
ax2.hist(df_age['AGE'], bins=30, color='coral', edgecolor='black', alpha=0.7)
ax2.axvline(df_age['AGE'].mean(), color='red', linestyle='--', linewidth=2, 
            label=f"Mean: {df_age['AGE'].mean():.1f} years")
ax2.axvline(df_age['AGE'].median(), color='blue', linestyle='--', linewidth=2,
            label=f"Median: {df_age['AGE'].median():.1f} years")
ax2.set_xlabel('Age at Dismantling (years)', fontweight='bold')
ax2.set_ylabel('Number of Ships', fontweight='bold')
ax2.set_title('Ship Age at Dismantling', fontweight='bold', fontsize=14)
ax2.legend()

plt.tight_layout()
plt.show()

print(f"\nAverage ship age at dismantling: {df_age['AGE'].mean():.1f} years")
print(f"Most common ship type: {type_counts.index[0]} ({type_counts.iloc[0]:,} ships)")

In [ ]:
# ============================================================================
# OWNER ANALYSIS - BENEFICIAL OWNERS
# ============================================================================

print("\n" + "="*80)
print("ANALYSIS 4: BENEFICIAL OWNERS OVERVIEW")
print("="*80)

# Filter to records with owner data
df_owners = df[df['OWNER'].notna()].copy()
print(f"\nRecords with owner data: {len(df_owners):,} ({len(df_owners)/len(df)*100:.1f}%)")

# Top beneficial owners
top_owners = df_owners['OWNER'].value_counts().head(20)

print("\nTop 20 Beneficial Owners by Volume:")
print("-" * 60)
for i, (owner, count) in enumerate(top_owners.items(), 1):
    print(f"{i:2d}. {owner[:50]:50s} {count:4d} ships")

In [ ]:
# ANALYSIS 5: Owner Countries
# ============================================================================

print("\n" + "="*80)
print("ANALYSIS 5: OWNER COUNTRIES")
print("="*80)

df_owner_country = df[df['OWNER_COUNTRY'].notna()].copy()
print(f"\nRecords with owner country data: {len(df_owner_country):,}")

owner_country_counts = df_owner_country['OWNER_COUNTRY'].value_counts().head(15)

fig, ax = plt.subplots(figsize=(12, 8))

colors = plt.cm.Blues(np.linspace(0.3, 0.9, len(owner_country_counts)))
bars = ax.barh(range(len(owner_country_counts)), owner_country_counts.values, color=colors, edgecolor='black')
ax.set_yticks(range(len(owner_country_counts)))
ax.set_yticklabels(owner_country_counts.index)
ax.set_xlabel('Number of Ships Owned', fontweight='bold')
ax.set_title('Beneficial Owner Countries', fontweight='bold', fontsize=14)
ax.invert_yaxis()

for i, count in enumerate(owner_country_counts.values):
    pct = count / len(df_owner_country) * 100
    ax.text(count + 20, i, f'{count:,} ({pct:.1f}%)', va='center', fontsize=10)

plt.tight_layout()
plt.show()

print("\nTop owner countries:")
for country, count in owner_country_counts.head(10).items():
    print(f"  {country}: {count:,} ships ({count/len(df_owner_country)*100:.1f}%)")

In [ ]:
# ANALYSIS 6: Owner Country to Destination Country Flow
# ============================================================================

print("\n" + "="*80)
print("ANALYSIS 6: OWNER COUNTRY TO DESTINATION FLOW")
print("="*80)

# Filter to records with both owner country and destination
df_flow = df[(df['OWNER_COUNTRY'].notna()) & (df['COUNTRY'].notna())].copy()
print(f"\nRecords with both owner country and destination: {len(df_flow):,}")

# Create flow matrix
flow = df_flow.groupby(['OWNER_COUNTRY', 'COUNTRY']).size().reset_index(name='COUNT')
flow_pivot = flow.pivot_table(index='OWNER_COUNTRY', columns='COUNTRY', values='COUNT', fill_value=0)

# Top owner countries sending to South Asia
south_asia_dest = ['India', 'Bangladesh', 'Pakistan']
south_asia_cols = [c for c in flow_pivot.columns if c in south_asia_dest]

if south_asia_cols:
    flow_pivot['TO_SOUTH_ASIA'] = flow_pivot[south_asia_cols].sum(axis=1)
    flow_pivot['TOTAL'] = flow_pivot.drop(columns=['TO_SOUTH_ASIA']).sum(axis=1)
    flow_pivot['SOUTH_ASIA_PCT'] = (flow_pivot['TO_SOUTH_ASIA'] / flow_pivot['TOTAL'] * 100).round(1)
    
    top_to_south_asia = flow_pivot[['TO_SOUTH_ASIA', 'TOTAL', 'SOUTH_ASIA_PCT']].sort_values('TO_SOUTH_ASIA', ascending=False).head(15)
    
    print("\nOwner countries by ships sent to South Asian yards:")
    print("-" * 70)
    print(f"{'Owner Country':<25} {'To South Asia':>15} {'Total':>10} {'%':>10}")
    print("-" * 70)
    for country, row in top_to_south_asia.iterrows():
        print(f"{country:<25} {int(row['TO_SOUTH_ASIA']):>15,} {int(row['TOTAL']):>10,} {row['SOUTH_ASIA_PCT']:>9.1f}%")

In [ ]:
# ANALYSIS 7: European Owners Analysis
# ============================================================================

print("\n" + "="*80)
print("ANALYSIS 7: EUROPEAN OWNER PATTERNS")
print("="*80)

# EU/European countries
european_countries = [
    'Germany', 'Greece', 'Norway', 'Denmark', 'Netherlands', 'Belgium',
    'United Kingdom', 'UK', 'France', 'Italy', 'Sweden', 'Finland',
    'Switzerland', 'Austria', 'Poland', 'Spain', 'Portugal', 'Cyprus',
    'Malta', 'Luxembourg', 'Ireland'
]

df_european = df_flow[df_flow['OWNER_COUNTRY'].isin(european_countries)].copy()
print(f"\nShips owned by European entities: {len(df_european):,}")

# Where do European-owned ships go?
european_destinations = df_european['COUNTRY'].value_counts()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Which European countries own the most scrapped ships
euro_owners = df_european['OWNER_COUNTRY'].value_counts().head(10)
ax1.barh(range(len(euro_owners)), euro_owners.values, color='navy', alpha=0.8)
ax1.set_yticks(range(len(euro_owners)))
ax1.set_yticklabels(euro_owners.index)
ax1.set_xlabel('Number of Ships', fontweight='bold')
ax1.set_title('European Countries by Ships Scrapped', fontweight='bold', fontsize=14)
ax1.invert_yaxis()

# Where do European ships end up?
euro_dest = european_destinations.head(8)
colors = ['#d62728' if d in south_asia_dest else '#1f77b4' for d in euro_dest.index]
ax2.barh(range(len(euro_dest)), euro_dest.values, color=colors, alpha=0.8)
ax2.set_yticks(range(len(euro_dest)))
ax2.set_yticklabels(euro_dest.index)
ax2.set_xlabel('Number of Ships', fontweight='bold')
ax2.set_title('Destinations for European-Owned Ships\n(Red = South Asia)', fontweight='bold', fontsize=14)
ax2.invert_yaxis()

plt.tight_layout()
plt.show()

# Calculate percentage going to South Asia
euro_to_south_asia = df_european[df_european['COUNTRY'].isin(south_asia_dest)].shape[0]
print(f"\nEuropean-owned ships sent to South Asian yards:")
print(f"  {euro_to_south_asia:,} out of {len(df_european):,} ({euro_to_south_asia/len(df_european)*100:.1f}%)")

In [ ]:
# ANALYSIS 8: Companies of Interest - High Volume
# ============================================================================

print("\n" + "="*80)
print("ANALYSIS 8: COMPANIES OF INTEREST - HIGH VOLUME")
print("="*80)
print("\nThese beneficial owners have sent the most ships to dismantling.")
print("This is provided for research and transparency purposes.")

# Top 10 owners with their preferred destinations
top_10_owners = df_owners['OWNER'].value_counts().head(10).index.tolist()

print("\n" + "=" * 80)

for owner in top_10_owners:
    owner_data = df_owners[df_owners['OWNER'] == owner]
    total = len(owner_data)
    destinations = owner_data['COUNTRY'].value_counts().head(3)
    
    owner_country = owner_data['OWNER_COUNTRY'].mode()
    owner_country_str = owner_country.iloc[0] if len(owner_country) > 0 else 'Unknown'
    
    print(f"\n{owner[:60]}")
    print(f"  Based in: {owner_country_str}")
    print(f"  Ships scrapped: {total}")
    print(f"  Top destinations:")
    for dest, count in destinations.items():
        pct = count/total*100
        print(f"    - {dest}: {count} ({pct:.0f}%)")

In [ ]:
# ANALYSIS 9: Companies of Interest - Tail Analysis (Bottom 50)
# ============================================================================

print("\n" + "="*80)
print("ANALYSIS 9: COMPANIES OF INTEREST - TAIL ANALYSIS")
print("="*80)
print("\nThese are beneficial owners appearing only once or twice in the dataset.")
print("Single-ship owners may warrant further review for research purposes.")
print("This can indicate shell companies or last-minute ownership transfers.")

# Get owners with only 1-2 ships
owner_counts = df_owners['OWNER'].value_counts()
single_ship_owners = owner_counts[owner_counts <= 2]

print(f"\nTotal beneficial owners in dataset: {len(owner_counts):,}")
print(f"Owners with only 1-2 ships: {len(single_ship_owners):,} ({len(single_ship_owners)/len(owner_counts)*100:.1f}%)")

# Show last 50 (tail)
tail_50 = owner_counts.tail(50)

print("\n" + "-" * 80)
print("LAST 50 BENEFICIAL OWNERS (by frequency):")
print("-" * 80)
print(f"{'#':<4} {'Beneficial Owner':<50} {'Ships':>6}")
print("-" * 80)

for i, (owner, count) in enumerate(tail_50.items(), 1):
    print(f"{i:<4} {owner[:50]:<50} {count:>6}")

In [ ]:
# ANALYSIS 10: Deep Dive on Tail Owners
# ============================================================================

print("\n" + "="*80)
print("ANALYSIS 10: TAIL OWNER DETAILS")
print("="*80)
print("\nDetailed view of the last 50 beneficial owners.")
print("Includes ship names, destinations, and owner countries.")

# Get the last 50 owners
tail_50_names = owner_counts.tail(50).index.tolist()

# Get their ship details
tail_ships = df_owners[df_owners['OWNER'].isin(tail_50_names)][[
    'YEAR', 'NAME', 'TYPE', 'OWNER', 'OWNER_COUNTRY', 'COUNTRY', 'LAST_FLAG'
]].sort_values(['OWNER', 'YEAR'])

print(f"\nShips from tail 50 owners: {len(tail_ships)}")
print("\n" + "-" * 120)

# Group by owner and show details
for owner in tail_50_names[:25]:  # Show first 25 of the tail
    owner_ships = tail_ships[tail_ships['OWNER'] == owner]
    print(f"\nOWNER: {owner}")
    for _, ship in owner_ships.iterrows():
        print(f"  {int(ship['YEAR'])} | {str(ship['NAME'])[:30]:<30} | {str(ship['TYPE'])[:15]:<15} | Flag: {str(ship['LAST_FLAG'])[:15]:<15} | Dest: {ship['COUNTRY']}")

In [ ]:
# ANALYSIS 11: Tail Owner Statistics
# ============================================================================

print("\n" + "="*80)
print("ANALYSIS 11: TAIL OWNER PATTERNS")
print("="*80)

# Statistics on single-ship owners
single_ship_data = df_owners[df_owners['OWNER'].isin(single_ship_owners.index)]

print(f"\nAnalyzing {len(single_ship_data):,} ships from {len(single_ship_owners):,} single/dual-ship owners")

# Where do single-ship owner vessels go?
single_dest = single_ship_data['COUNTRY'].value_counts().head(10)
single_to_sa = single_ship_data[single_ship_data['COUNTRY'].isin(south_asia_dest)].shape[0]

print(f"\nDestinations for single-ship owner vessels:")
for dest, count in single_dest.items():
    pct = count / len(single_ship_data) * 100
    print(f"  {dest}: {count:,} ({pct:.1f}%)")

print(f"\nSouth Asia total: {single_to_sa:,} ({single_to_sa/len(single_ship_data)*100:.1f}%)")

# Owner countries of single-ship owners
single_owner_countries = single_ship_data['OWNER_COUNTRY'].value_counts().head(10)
print(f"\nOwner countries (single-ship owners):")
for country, count in single_owner_countries.items():
    print(f"  {country}: {count:,}")

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Destinations
colors1 = ['#d62728' if d in south_asia_dest else '#1f77b4' for d in single_dest.index]
ax1.barh(range(len(single_dest)), single_dest.values, color=colors1, alpha=0.8)
ax1.set_yticks(range(len(single_dest)))
ax1.set_yticklabels(single_dest.index)
ax1.set_xlabel('Number of Ships', fontweight='bold')
ax1.set_title('Where Single-Ship Owners Send Vessels\n(Red = South Asia)', fontweight='bold', fontsize=14)
ax1.invert_yaxis()

# Owner countries
ax2.barh(range(len(single_owner_countries)), single_owner_countries.values, color='teal', alpha=0.8)
ax2.set_yticks(range(len(single_owner_countries)))
ax2.set_yticklabels(single_owner_countries.index)
ax2.set_xlabel('Number of Ships', fontweight='bold')
ax2.set_title('Single-Ship Owner Countries', fontweight='bold', fontsize=14)
ax2.invert_yaxis()

plt.tight_layout()
plt.show()

In [ ]:
# ANALYSIS 12: Trends Over Time
# ============================================================================

print("\n" + "="*80)
print("ANALYSIS 12: TRENDS OVER TIME")
print("="*80)

# South Asia percentage by year
yearly_south_asia = df.groupby('YEAR').apply(
    lambda x: (x['COUNTRY'].isin(south_asia_dest).sum() / len(x) * 100) if len(x) > 0 else 0
)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# South Asia percentage over time
ax1.plot(yearly_south_asia.index, yearly_south_asia.values, marker='o', linewidth=2, markersize=8, color='darkred')
ax1.fill_between(yearly_south_asia.index, yearly_south_asia.values, alpha=0.3, color='red')
ax1.set_xlabel('Year', fontweight='bold')
ax1.set_ylabel('% to South Asian Yards', fontweight='bold')
ax1.set_title('Share of Ships Going to South Asian Yards', fontweight='bold', fontsize=14)
ax1.set_ylim([0, 100])
ax1.grid(True, alpha=0.3)
ax1.axhline(y=80, color='gray', linestyle='--', alpha=0.5, label='80% line')

# Top destinations over time (stacked area)
top_dest = df['COUNTRY'].value_counts().head(5).index.tolist()
yearly_dest = df[df['COUNTRY'].isin(top_dest)].groupby(['YEAR', 'COUNTRY']).size().unstack(fill_value=0)

yearly_dest.plot(kind='area', stacked=True, ax=ax2, alpha=0.8)
ax2.set_xlabel('Year', fontweight='bold')
ax2.set_ylabel('Number of Ships', fontweight='bold')
ax2.set_title('Top 5 Destinations Over Time', fontweight='bold', fontsize=14)
ax2.legend(loc='upper right')

plt.tight_layout()
plt.show()

print(f"\nSouth Asia share: {yearly_south_asia.iloc[0]:.1f}% in {int(yearly_south_asia.index[0])} → {yearly_south_asia.iloc[-1]:.1f}% in {int(yearly_south_asia.index[-1])}")

In [ ]:
# FINAL SUMMARY
# ============================================================================

print("="*80)
print("SUMMARY")
print("="*80)

south_asia_pct = df[df['COUNTRY'].isin(south_asia_dest)].shape[0] / len(df) * 100

print(f"""
SHIP BREAKING DATA ANALYSIS
===========================

DATASET OVERVIEW:
- {len(df):,} ships dismantled globally (2012-2024)
- Average ship age at scrapping: {df_age['AGE'].mean():.0f} years
- Most common type: {df['TYPE'].value_counts().index[0]}

GEOGRAPHIC PATTERNS:
- {south_asia_pct:.1f}% of ships go to South Asian yards
- Top destination: {df['COUNTRY'].value_counts().index[0]}

BENEFICIAL OWNER ANALYSIS:
- Owner data available for {df['OWNER'].notna().sum():,} ships ({df['OWNER'].notna().sum()/len(df)*100:.1f}%)
- Total unique owners: {df_owners['OWNER'].nunique():,}
- Single/dual-ship owners: {len(single_ship_owners):,} ({len(single_ship_owners)/len(owner_counts)*100:.1f}%)

KEY OBSERVATIONS:
- High concentration of ships going to a few destination countries
- Large number of single-ship owners may indicate ownership transfers
- Owner country data enables tracking of cross-border ship flows

This analysis is provided for research and transparency purposes.
""")